In [2]:
import pandas as pd
import json
import ast
import numpy as np



In [3]:
data = pd.read_csv("/Users/taeeunkwon/Downloads/vscode_projects/data/final_data_for_svr.csv")
data.head()

,Unnamed: 0,_id,duration,likes,speakers,subtitle_languages,summary,topics,title,transcript,...,Curious,Informed,Impressed,Hopeful,Amused,Fulfilled,Surprised,tb_score,vd_score,Unnamed: 0.1
0,0,21,992,17000,"[{'name': 'Mena Trott', 'occupation': 'Blogger...","[{'name': 'English', 'code': 'en'}, {'name': '...","The founding mother of the blog revolution, Mo...","[{'id': '4', 'name': 'culture'}, {'id': '13', ...",Meet the founder of the blog revolution,"Over the past couple of days, as I've been pre...",...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.133855,0.9996,0
1,1,1,957,110000,"[{'name': 'Al Gore', 'occupation': 'Climate ad...","[{'name': 'Greek', 'code': 'el'}, {'name': 'Gu...",With the same humor and humanity he exuded in ...,"[{'id': '3', 'name': 'climate change'}, {'id':...",Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",...,0.75,0.00,0.0,0.0,0.0,0.0,0.0,0.131762,0.9997,1
2,2,7,1266,60000,"[{'name': 'David Pogue', 'occupation': 'Techno...","[{'name': 'Greek', 'code': 'el'}, {'name': 'En...",New York Times columnist David Pogue takes aim...,"[{'id': '10', 'name': 'technology'}, {'id': '3...",Simplicity sells,"(Music: ""The Sound of Silence,"" Simon &amp; Ga...",...,0.90,0.80,0.6,0.0,0.0,0.0,0.0,0.155493,0.9999,2
3,3,47,1126,80000,"[{'name': 'David Deutsch', 'occupation': 'Phys...","[{'name': 'Greek', 'code': 'el'}, {'name': 'En...",Legendary scientist David Deutsch puts theoret...,"[{'id': '3', 'name': 'climate change'}, {'id':...",Chemical scum that dream of distant quasars,We've been told to go out on a limb and say so...,...,0.00,0.65,0.0,0.0,0.0,0.0,0.0,0.066814,0.9990,3
4,4,55,1524,14000,"[{'name': 'Jehane Noujaim', 'occupation': 'Fil...","[{'name': 'English', 'code': 'en'}, {'name': '...",Jehane Noujaim unveils her 2006 TED Prize wish...,"[{'id': '4', 'name': 'culture'}, {'id': '6', '...",My wish: A global day of film,I can't help but this wish: to think about whe...,...,0.75,0.00,0.0,0.0,0.0,0.0,0.0,0.127882,0.9995,4


In [4]:
# Function to safely parse the structured string into a dictionary and extract the occupation
def extract_occupation(speaker_info):
    try:
        # Convert the string representation of list-dictionaries into actual list of dictionaries
        speaker_list = ast.literal_eval(speaker_info)
        # Extract the first speaker's occupation (assuming at least one speaker per talk)
        occupation = speaker_list[0]['occupation']
        return occupation
    except Exception as e:
        # In case of any error (e.g., malformed string, missing keys), return NaN
        return np.nan

# Apply the function to the 'speakers' column to create the new 'occupation' column
data['occupation'] = data['speakers'].apply(extract_occupation)

# Display the first few rows to verify the new column
data[['speakers', 'occupation']].head()

,speakers,occupation
0,"[{'name': 'Mena Trott', 'occupation': 'Blogger...","Blogger; cofounder, Six Apart"
1,"[{'name': 'Al Gore', 'occupation': 'Climate ad...",Climate advocate
2,"[{'name': 'David Pogue', 'occupation': 'Techno...",Technology columnist
3,"[{'name': 'David Deutsch', 'occupation': 'Phys...","Physicist, author"
4,"[{'name': 'Jehane Noujaim', 'occupation': 'Fil...",Filmmaker


Thought about multiple way to select or reduce variables but choose **Top N Occupations + 'Other' Category** method because we are trying to build an explanatory model

In [5]:
# Splitting the occupations by common delimiters and flattening the list
occupation_list = data['occupation'].dropna().str.split(';|,').explode().str.strip()

# Counting the frequency of each unique occupation
occupation_counts = occupation_list.value_counts()

# Getting the top 50 occupations
top_50_occupations = occupation_counts.head(50)
top_50_occupations


occupation
Educator               906
author                 129
Writer                 118
                       111
Author                 102
Journalist              86
Artist                  83
Entrepreneur            76
activist                74
Designer                66
entrepreneur            57
writer                  50
Neuroscientist          48
Architect               47
Psychologist            46
Economist               42
Inventor                42
Musician                39
Physicist               35
Filmmaker               34
Technologist            34
Activist                33
educator                32
Photographer            30
Biologist               29
Actor                   27
Philosopher             27
Physician               26
Poet                    25
professor               24
inventor                23
Roboticist              23
Engineer                21
futurist                21
Social entrepreneur     21
Historian               20
Sleep scientist  

In [6]:
# Split the occupations by semicolon or comma and then normalize them
occupation_series = data['occupation'].str.split(';|,', expand=True).stack().str.strip()
occupation_counts = occupation_series.value_counts()

# Get the top 50 occupations
top_50_occupations = occupation_counts.head(50).index.tolist()

# Display the top 50 occupations and their counts
top_50_occupations, occupation_counts.head(50)


(['Educator',
  'author',
  'Writer',
  '',
  'Author',
  'Journalist',
  'Artist',
  'Entrepreneur',
  'activist',
  'Designer',
  'entrepreneur',
  'writer',
  'Neuroscientist',
  'Architect',
  'Psychologist',
  'Economist',
  'Inventor',
  'Musician',
  'Physicist',
  'Filmmaker',
  'Technologist',
  'Activist',
  'educator',
  'Photographer',
  'Biologist',
  'Actor',
  'Philosopher',
  'Physician',
  'Poet',
  'professor',
  'inventor',
  'Roboticist',
  'Engineer',
  'futurist',
  'Social entrepreneur',
  'Historian',
  'Sleep scientist',
  'artist',
  'filmmaker',
  'Singer-songwriter',
  'CEO',
  'Singer',
  'Astrophysicist',
  'Computer scientist',
  'designer',
  'Scientist',
  'Comedian',
  'Politician',
  'musician',
  'producer'],
 Educator               906
 author                 129
 Writer                 118
                        111
 Author                 102
 Journalist              86
 Artist                  83
 Entrepreneur            76
 activist            

In [7]:
# Normalize all occupation entries to lowercase to merge duplicates
occupation_series_normalized = occupation_series.str.lower()
occupation_counts_normalized = occupation_series_normalized.value_counts()

# Get the new top 50 occupations
new_top_50_occupations = occupation_counts_normalized.head(50).index.tolist()

# Display the new top 50 occupations and their counts
new_top_50_occupations, occupation_counts_normalized.head(50)


(['educator',
  'author',
  'writer',
  'entrepreneur',
  '',
  'activist',
  'artist',
  'journalist',
  'designer',
  'inventor',
  'musician',
  'filmmaker',
  'architect',
  'neuroscientist',
  'psychologist',
  'economist',
  'technologist',
  'professor',
  'social entrepreneur',
  'physicist',
  'photographer',
  'philosopher',
  'poet',
  'actor',
  'biologist',
  'physician',
  'engineer',
  'futurist',
  'roboticist',
  'computer scientist',
  'researcher',
  'comedian',
  'historian',
  'producer',
  'singer',
  'singer-songwriter',
  'scientist',
  'director',
  'philanthropist',
  'investor',
  'sleep scientist',
  'ceo',
  'lawyer',
  'anthropologist',
  'social psychologist',
  'environmentalist',
  'politician',
  'astrophysicist',
  'novelist',
  'editor'],
 educator               938
 author                 231
 writer                 168
 entrepreneur           133
                        111
 activist               107
 artist                 102
 journalist        

In [10]:
# Simplify the column creation by verifying step by step

# Normalize and prepare the occupations in a set
top_occupations_set = set(new_top_50_occupations)

# Initialize columns for each occupation and 'Other'
for occupation in top_occupations_set:
    data[f'occupation_{occupation}'] = 0
data['occupation_Other'] = 0

# Map occupations to columns
def assign_occupations(row):
    row_occupations = set(occ.strip().lower() for occ in str(row['occupation']).split(';'))
    matched = False
    for occupation in top_occupations_set:
        if occupation in row_occupations:
            row[f'occupation_{occupation}'] = 1
            matched = True
    if not matched:
        row['occupation_Other'] = 1
    return row

# Apply the function across the dataset
data = data.apply(assign_occupations, axis=1)

# Verify the creation of the occupation columns and the 'Other' category
#data[[f'occupation_{occupation}' for occupation in new_top_50_occupations] + ['occupation_Other']].head()

# Show all the columns
data.columns

Index(['Unnamed: 0', '_id', 'duration', 'likes', 'speakers',
       'subtitle_languages', 'summary', 'topics', 'title', 'transcript',
       'views', 'recorded_date', 'published_date', 'speakers_list', 'sl_list',
       'topics_list', 'processed_transcript', 'Inspired', 'Moved', 'Sad',
       'Curious', 'Informed', 'Impressed', 'Hopeful', 'Amused', 'Fulfilled',
       'Surprised', 'tb_score', 'vd_score', 'Unnamed: 0.1', 'occupation',
       'occupation_', 'occupation_writer', 'occupation_futurist',
       'occupation_astrophysicist', 'occupation_singer', 'occupation_author',
       'occupation_biologist', 'occupation_physicist', 'occupation_architect',
       'occupation_poet', 'occupation_entrepreneur', 'occupation_novelist',
       'occupation_journalist', 'occupation_physician', 'occupation_lawyer',
       'occupation_director', 'occupation_educator', 'occupation_inventor',
       'occupation_philosopher', 'occupation_professor', 'occupation_comedian',
       'occupation_environment